# CpGPTGrimAge3

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.CpGPTGrimAge3)

class CpGPTGrimAge3(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Scales an array based on the median and standard deviation.
        """
        median = torch.tensor(self.preprocess_dependencies[0], device=x.device, dtype=x.dtype)
        std = torch.tensor(self.preprocess_dependencies[1], device=x.device, dtype=x.dtype)
        x = (x - median) / std
        return x

    def postprocess(self, x):
        """
        Converts from a Cox parameter to age in units of years.
        """
        cox_mean = -6.60427912e-16
        cox_std = 1.72004470842156
        age_mean = 58.8234007654456
        age_std = 13.091231557630831

        # Normalize
        x = (x - cox_mean) / cox_std

        # Scale
        x = (x * age_std) + age_mean

        return x



In [3]:
model = pya.models.CpGPTGrimAge3()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'cpgptgrimage3'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "de Lima Camillo, Lucas Paulo, et al. \"CpGPT: a foundation model for DNA methylation.\" bioRxiv (2024): 2024-10."
model.metadata["doi"] = "https://doi.org/10.1101/2024.10.24.619766"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_grimage3_weights.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_pcgrimage3_weights.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_mean.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/input_scaler_scale.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/pca_scaler_mean.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/pca_scaler_scale.npy",
    "https://pyaging.s3.amazonaws.com/supporting_files/cpgpt_grimage3_dependencies/cpgpt_pcgrimage3_pca_components.npy",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Data found in ./cpgpt_grimage3_weights.csv
|-----------> Data found in ./cpgpt_pcgrimage3_weights.csv
|-----------> Data found in ./input_scaler_mean.npy
|-----------> Data found in ./input_scaler_scale.npy
|-----------> Data found in ./pca_scaler_mean.npy
|-----------> Data found in ./pca_scaler_scale.npy
|-----------> Data found in ./cpgpt_pcgrimage3_pca_components.npy


## Load features

#### From CSV

In [6]:
df = pd.read_csv('cpgpt_grimage3_weights.csv')
model.features = df['feature'].tolist()

In [7]:
df.head()

,feature,coefficient
0,age,0.671578
1,grimage2timp1,0.233130
2,grimage2packyrs,0.401592
3,grimage2logcrp,0.297703
4,grimage2b2m,0.025120


## Load weights into base model

#### Linear model

In [8]:
weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [10]:
scale_mean = np.load('input_scaler_mean.npy')
scale_std = np.load('input_scaler_scale.npy')

model.reference_values = None

## Load preprocess and postprocess objects

In [11]:
model.preprocess_name = 'scale'
model.preprocess_dependencies = [scale_mean, scale_std]

In [12]:
model.postprocess_name = 'cox_to_years'
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'de Lima Camillo, Lucas Paulo, et al. "CpGPT: a foundation model '
             'for DNA methylation." bioRxiv (2024): 2024-10.',
 'clock_name': 'cpgptgrimage3',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1101/2024.10.24.619766',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2025}
reference_values: None
preprocess_name: 'scale'
preprocess_dependencies: [array([ 5.88234008e+01,  3.37892988e+04,  2.13030677e+01,  6.29543290e-02,
        1.71182803e+06,  3.48032879e+02,  8.07863072e+03,  8.40183045e+02,
       -4.62783972e-01, -2.81553566e-01,  2.08716775e-02, -2.03755490e-01,
       -3.31462601e-01, -1.99447326e-01, -2.86641682e-01,  9.69636186e-02,
        9.23084436e-02, -1.77190600e-01, -5.19144939e-01,  2.77395512e-01,
       -5.05034895e-02, -1.697

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-324.0034],
        [-153.8915],
        [-281.4972],
        [-202.1743],
        [ 498.1493],
        [ -59.9054],
        [-262.7733],
        [ -29.8534],
        [ -39.1590],
        [-334.5076]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: cpgpt_pcgrimage3_pca_components.npy
Deleted file: cpgpt_pcgrimage3_weights.csv
Deleted file: input_scaler_scale.npy
Deleted file: CalcAllPCClocks.RData
Deleted file: pca_scaler_mean.npy
Deleted file: pca_scaler_scale.npy
Deleted file: cpgpt_grimage3_weights.csv
Deleted file: input_scaler_mean.npy
